In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/code/baseline/UniverseNet/configs/universenet/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco.py')

root='../../../dataset/'

epoch = 'latest'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (1333,800) # Resize
cfg.data.test.test_mode = True
cfg.model.test_cfg.score_thr = 0.05

cfg.data.samples_per_gpu = 16

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/2023_05_17_18_36_39_GFL'

cfg.model.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/code/baseline/UniverseNet/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: ./work_dirs/2023_05_17_18_36_39_GFL/latest.pth


2023-05-17 14:42:26,941 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.0 is upgraded to version 2.
2023-05-17 14:42:26,943 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.1 is upgraded to version 2.
2023-05-17 14:42:26,945 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.2 is upgraded to version 2.
2023-05-17 14:42:26,951 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.0 is upgraded to version 2.
2023-05-17 14:42:26,952 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.1 is upgraded to version 2.
2023-05-17 14:42:26,954 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.2 is upgraded to version 2.
2023-05-17 14:42:26,958 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.0 is upgraded to version 2.
2023-05-17 14:42:26,959 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.1 is upgraded to version 2.
2023-05-17 14:42:26,961 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.2 is upgraded to version 2.
2023-05-17 14:42:26

In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 7.0 task/s, elapsed: 694s, ETA:     0s

In [6]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.23156068 495.20093 595.97046 551.58575 644...,test/0000.jpg
1,0 0.3851502 638.32806 648.2826 685.13116 842.2...,test/0001.jpg
2,0 0.8441453 879.4398 464.67267 1024.0 637.0589...,test/0002.jpg
3,0 0.26585767 0.88066405 0.0 1024.0 1024.0 0 0....,test/0003.jpg
4,0 0.6532128 186.6841 248.41743 876.11084 778.9...,test/0004.jpg


In [7]:
submission[['PredictionString']]

,PredictionString
0,0 0.23156068 495.20093 595.97046 551.58575 644...
1,0 0.3851502 638.32806 648.2826 685.13116 842.2...
2,0 0.8441453 879.4398 464.67267 1024.0 637.0589...
3,0 0.26585767 0.88066405 0.0 1024.0 1024.0 0 0....
4,0 0.6532128 186.6841 248.41743 876.11084 778.9...
...,...
4866,0 0.80204207 284.1005 347.3707 668.573 686.155...
4867,3 0.8884071 415.38983 313.71664 780.7212 703.6...
4868,0 0.25688747 546.6752 539.4515 674.9827 764.63...
4869,0 0.88763416 0.0 237.63797 1017.7991 928.9441 ...
